In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats


In [10]:
def continuous_var_summary( x ):
    
    # freq and missings
    n_total = x.shape[0]
    n_miss = x.isna().sum()
    perc_miss = n_miss * 100 / n_total
    
    # outliers - iqr
    q1 = x.quantile(0.25)
    q3 = x.quantile(0.75)
    iqr = q3 - q1
    lc_iqr = q1 - 1.5 * iqr
    uc_iqr = q3 + 1.5 * iqr
    
    
    return pd.Series( [ x.dtype, x.nunique(), n_total, x.count(), n_miss, perc_miss,
                       x.sum(), x.mean(), x.std(), x.var(), 
                       lc_iqr, uc_iqr, 
                       x.min(), x.quantile(0.01), x.quantile(0.05), x.quantile(0.10), 
                       x.quantile(0.25), x.quantile(0.5), x.quantile(0.75), 
                       x.quantile(0.90), x.quantile(0.95), x.quantile(0.99), x.max() ], 
                     
                    index = ['dtype', 'cardinality', 'n_tot', 'n', 'nmiss', 'perc_miss',
                             'sum', 'mean', 'std', 'var',
                        'lc_iqr', 'uc_iqr',
                        'min', 'p1', 'p5', 'p10', 'p25', 'p50', 'p75', 'p90', 'p95', 'p99', 'max']) 

In [11]:
# Create Data audit Report for categorical variables
def categorical_var_summary( x ):
    Mode = x.value_counts().sort_values(ascending = False)[0:1].reset_index()
    return pd.Series([x.count(), x.isnull().sum(), Mode.iloc[0, 0], Mode.iloc[0, 1], 
                          round(Mode.iloc[0, 1] * 100 / x.count(), 2)], 
                     
                  index = ['N', 'NMISS', 'MODE', 'FREQ', 'PERCENT'])

In [12]:
# Missing value imputation for continuous variables
def missing_imputation( x, stats = 'mean' ):
    if (x.dtypes == 'float64') | (x.dtypes == 'int64'):
        x = x.fillna(x.mean()) if stats == 'mean' else x.fillna(x.median())
    return x

In [18]:
loan_data= pd.read_csv('E:/DataSetProject4/LoansData.csv')

# BUSINESS PROBLEM-1

In [19]:
loan_data.head(2)

,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
0,20000.0,20000.0,8.90%,36 months,debt_consolidation,14.90%,SC,MORTGAGE,6541.67,735-739,14.0,14272.0,2.0,< 1 year
1,19200.0,19200.0,12.12%,36 months,debt_consolidation,28.36%,TX,MORTGAGE,4583.33,715-719,12.0,11140.0,1.0,2 years


In [20]:
loan_data.columns

Index(['Amount.Requested', 'Amount.Funded.By.Investors', 'Interest.Rate',
       'Loan.Length', 'Loan.Purpose', 'Debt.To.Income.Ratio', 'State',
       'Home.Ownership', 'Monthly.Income', 'FICO.Range', 'Open.CREDIT.Lines',
       'Revolving.CREDIT.Balance', 'Inquiries.in.the.Last.6.Months',
       'Employment.Length'],
      dtype='object')

In [22]:
loan_data.rename(columns={'Amount.Requested':'amount_requested', 'Amount.Funded.By.Investors':'amount_funded_by_investors', 'Interest.Rate':'interest_rate',
       'Loan.Length':'loan_length', 'Loan.Purpose':'loan_purpose', 'Debt.To.Income.Ratio':'debt_to_income_ratio', 'State':'state',
       'Home.Ownership':'home_ownership','Monthly.Income':'monthly_income', 'FICO.Range':'FICO_Range', 'Open.CREDIT.Lines':'open_CREDIT_lines',
       'Revolving.CREDIT.Balance':'revolving_CREDIT_balance', 'Inquiries.in.the.Last.6.Months':'inquiries_last_6Months',
       'Employment.Length':'employment_length'},inplace=True)
loan_data.head(2)

,amount_requested,amount_funded_by_investors,interest_rate,loan_length,loan_purpose,debt_to_income_ratio,state,home_ownership,monthly_income,FICO_Range,open_CREDIT_lines,revolving_CREDIT_balance,inquiries_last_6Months,employment_length
0,20000.0,20000.0,8.90%,36 months,debt_consolidation,14.90%,SC,MORTGAGE,6541.67,735-739,14.0,14272.0,2.0,< 1 year
1,19200.0,19200.0,12.12%,36 months,debt_consolidation,28.36%,TX,MORTGAGE,4583.33,715-719,12.0,11140.0,1.0,2 years


In [23]:
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   amount_requested            2499 non-null   float64
 1   amount_funded_by_investors  2499 non-null   float64
 2   interest_rate               2498 non-null   object 
 3   loan_length                 2500 non-null   object 
 4   loan_purpose                2500 non-null   object 
 5   debt_to_income_ratio        2499 non-null   object 
 6   state                       2500 non-null   object 
 7   home_ownership              2499 non-null   object 
 8   monthly_income              2499 non-null   float64
 9   FICO_Range                  2498 non-null   object 
 10  open_CREDIT_lines           2497 non-null   float64
 11  revolving_CREDIT_balance    2497 non-null   float64
 12  inquiries_last_6Months      2497 non-null   float64
 13  employment_length           2423 

In [24]:
loan_data.isnull().sum()

amount_requested               1
amount_funded_by_investors     1
interest_rate                  2
loan_length                    0
loan_purpose                   0
debt_to_income_ratio           1
state                          0
home_ownership                 1
monthly_income                 1
FICO_Range                     2
open_CREDIT_lines              3
revolving_CREDIT_balance       3
inquiries_last_6Months         3
employment_length             77
dtype: int64

In [25]:
loan_data.interest_rate = loan_data.interest_rate.str.replace('%','')
loan_data.debt_to_income_ratio = loan_data.debt_to_income_ratio.str.replace('%','')


In [26]:
loan_data.loan_length = loan_data.loan_length.str.replace("months", '')

In [27]:
loan_data["interest_rate"]= loan_data["interest_rate"].astype(float)
loan_data["debt_to_income_ratio"]= loan_data["debt_to_income_ratio"].astype(float)
loan_data["loan_length"]= loan_data["loan_length"].astype(float)


In [28]:
#continous and categorical var
numeric_columns = loan_data.select_dtypes(include = ['float64', 'int64']).columns
object_columns = loan_data.select_dtypes(include = ['object']).columns

In [29]:
numeric_columns

Index(['amount_requested', 'amount_funded_by_investors', 'interest_rate',
       'loan_length', 'debt_to_income_ratio', 'monthly_income',
       'open_CREDIT_lines', 'revolving_CREDIT_balance',
       'inquiries_last_6Months'],
      dtype='object')

In [30]:
object_columns

Index(['loan_purpose', 'state', 'home_ownership', 'FICO_Range',
       'employment_length'],
      dtype='object')

In [31]:
loan_data_conti_vars = loan_data.select_dtypes(include = ['float64', 'int64'])
loan_data_cat = loan_data.select_dtypes(include = ['object'])

In [32]:
loan_data_conti_vars.apply( continuous_var_summary ).round(1)

,amount_requested,amount_funded_by_investors,interest_rate,loan_length,debt_to_income_ratio,monthly_income,open_CREDIT_lines,revolving_CREDIT_balance,inquiries_last_6Months
dtype,float64,float64,float64,float64,float64,float64,float64,float64,float64
cardinality,380,710,275,2,1669,632,29,2349,10
n_tot,2500,2500,2500,2500,2500,2500,2500,2500,2500
n,2499,2499,2498,2500,2499,2499,2497,2497,2497
nmiss,1,1,2,0,1,1,3,3,3
perc_miss,0.04,0.04,0.08,0.0,0.04,0.04,0.12,0.12,0.12
sum,31001250.0,29993933.09,32637.34,103152.0,38435.67,14216639.37,25152.0,38012292.0,2264.0
mean,12405.462185,12002.374186,13.065388,41.2608,15.38042,5688.931321,10.072887,15223.184622,0.906688
std,7802.933666,7746.767348,4.177201,9.930893,7.505052,3963.118185,4.507416,18281.015258,1.231149
var,60885773.795507,60012404.344627,17.449011,98.622632,56.325811,15706305.750123,20.316801,334195518.850116,1.515728


In [33]:
loan_data_cat.apply(categorical_var_summary).T

,N,NMISS,MODE,FREQ,PERCENT
loan_purpose,2500,0,debt_consolidation,1307,52.28
state,2500,0,CA,433,17.32
home_ownership,2499,1,MORTGAGE,1147,45.9
FICO_Range,2498,2,670-674,171,6.85
employment_length,2423,77,10+ years,653,26.95


#### OUTLIER TREATMENT

In [34]:
loan_data_conti_vars = loan_data_conti_vars.apply( lambda x: x.clip(lower = x.quantile(0.01),
                                                         upper = x.quantile(0.99)))

In [35]:
loan_data_conti_vars.apply(continuous_var_summary).round(1)

,amount_requested,amount_funded_by_investors,interest_rate,loan_length,debt_to_income_ratio,monthly_income,open_CREDIT_lines,revolving_CREDIT_balance,inquiries_last_6Months
dtype,float64,float64,float64,float64,float64,float64,float64,float64,float64
cardinality,375,691,265,2,1629,602,21,2325,6
n_tot,2500,2500,2500,2500,2500,2500,2500,2500,2500
n,2499,2499,2498,2500,2499,2499,2497,2497,2497
nmiss,1,1,2,0,1,1,3,3,3
perc_miss,0.04,0.04,0.08,0.0,0.04,0.04,0.12,0.12,0.12
sum,31010275.0,30008521.52,32624.7375,103152.0,38430.94,14015763.65,25105.0,36878653.0,2216.0
mean,12409.073629,12008.211893,13.060343,41.2608,15.378527,5608.54888,10.054065,14769.184221,0.887465
std,7797.780053,7738.184046,4.149235,9.930893,7.480489,3084.368635,4.379883,14767.130324,1.142032
var,60805373.751181,59879492.335294,17.21615,98.622632,55.957718,9513329.874068,19.183374,218068137.996594,1.304238


##### MISSING VALUE TREATMENT

In [36]:
loan_data_conti_vars.isna().sum() * 100 / loan_data_conti_vars.isna().count()

amount_requested              0.04
amount_funded_by_investors    0.04
interest_rate                 0.08
loan_length                   0.00
debt_to_income_ratio          0.04
monthly_income                0.04
open_CREDIT_lines             0.12
revolving_CREDIT_balance      0.12
inquiries_last_6Months        0.12
dtype: float64

In [37]:
loan_data_conti_vars = loan_data_conti_vars.apply(missing_imputation)

In [38]:
loan_data_conti_vars.isna().sum()

amount_requested              0
amount_funded_by_investors    0
interest_rate                 0
loan_length                   0
debt_to_income_ratio          0
monthly_income                0
open_CREDIT_lines             0
revolving_CREDIT_balance      0
inquiries_last_6Months        0
dtype: int64

In [39]:
loan_data_conti_vars.apply(continuous_var_summary).round(1)

,amount_requested,amount_funded_by_investors,interest_rate,loan_length,debt_to_income_ratio,monthly_income,open_CREDIT_lines,revolving_CREDIT_balance,inquiries_last_6Months
dtype,float64,float64,float64,float64,float64,float64,float64,float64,float64
cardinality,376,692,266,2,1630,603,22,2326,7
n_tot,2500,2500,2500,2500,2500,2500,2500,2500,2500
n,2500,2500,2500,2500,2500,2500,2500,2500,2500
nmiss,0,0,0,0,0,0,0,0,0
perc_miss,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sum,31022684.073629,30020529.731893,32650.858187,103152.0,38446.318527,14021372.19888,25135.162195,36922960.552663,2218.662395
mean,12409.073629,12008.211893,13.060343,41.2608,15.378527,5608.54888,10.054065,14769.184221,0.887465
std,7796.219717,7736.635635,4.147574,9.930893,7.478992,3083.751452,4.377253,14758.263838,1.141347
var,60781041.868928,59855530.953807,17.202371,98.622632,55.935326,9509523.019376,19.160345,217806351.516406,1.302672


#### a. Intrest rate is varied for different loan amounts

In [40]:
# Ho = no relationship
# Ha = there is a relationship among the variables
# CI = 95%
# p-value = 0.05
# perform the test
stats.pearsonr( loan_data_conti_vars.interest_rate , loan_data_conti_vars.amount_funded_by_investors)






(0.3356612554420613, 6.841366109528892e-67)

In [41]:
# business corelation
# interest rate is varied for different loan amounts

#### b. Loan length is directly effecting intrest rate.

In [42]:
# Ho = no relationship
# Ha = there is a relationship among the variables
# CI = 95%
# p-value = 0.05
# perform the test1
stats.pearsonr (loan_data_conti_vars.interest_rate,loan_data_conti_vars.loan_length)

(0.42315251053314384, 3.565010388635864e-109)

In [43]:
# business corelation
# loan length is directly effecting interest rate

##### c. Inrest rate varies for different purpose of loans

In [44]:
loan_data_cat.loan_purpose.nunique()

14

In [143]:
loan_merge1=loan_data_conti_vars.reset_index()
loan_merge2=loan_data_cat.reset_index()
loan_merge = pd.merge(left = loan_merge1,right = loan_merge2, on = 'index')
loan_merge.isnull().sum()


index                          0
amount_requested               0
amount_funded_by_investors     0
interest_rate                  0
loan_length                    0
debt_to_income_ratio           0
monthly_income                 0
open_CREDIT_lines              0
revolving_CREDIT_balance       0
inquiries_last_6Months         0
loan_purpose                   0
state                          0
home_ownership                 1
FICO_Range                     2
employment_length             77
dtype: int64

In [146]:
# data prep
a1 = loan_merge.loc[loan_merge.loan_purpose=='debt_consolidation','interest_rate']
a2 = loan_merge.loc[loan_merge.loan_purpose=='credit_card','interest_rate']
a3 = loan_merge.loc[loan_merge.loan_purpose=='other','interest_rate']
a4 = loan_merge.loc[loan_merge.loan_purpose=='moving','interest_rate']
a5 = loan_merge.loc[loan_merge.loan_purpose=='car','interest_rate']
a6 = loan_merge.loc[loan_merge.loan_purpose=='vacation','interest_rate']
a7 = loan_merge.loc[loan_merge.loan_purpose=='home_improvement','interest_rate']
a8 = loan_merge.loc[loan_merge.loan_purpose=='house','interest_rate']
a9 = loan_merge.loc[loan_merge.loan_purpose=='major_purchase','interest_rate']
a10 = loan_merge.loc[loan_merge.loan_purpose=='educational','interest_rate']
a11= loan_merge.loc[loan_merge.loan_purpose=='medical','interest_rate']
a12= loan_merge.loc[loan_merge.loan_purpose=='wedding','interest_rate']
a13 = loan_merge.loc[loan_merge.loan_purpose=='mall_business','interest_rate']
a14= loan_merge.loc[loan_merge.loan_purpose=='renewable_energy','interest_rate']











In [150]:
# Ho = no relationship
# Ha = there is a relationship among the variables
# CI = 95%
# p-value = 0.05
# perform the test
print( stats.f_oneway( a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11,a12,a14) )

F_onewayResult(statistic=8.043334227324289, pvalue=5.706099579457128e-15)


###### d. There is relationship between FICO scores and Home Ownership.

In [46]:
loan_data_cat

,loan_purpose,state,home_ownership,FICO_Range,employment_length
0,debt_consolidation,SC,MORTGAGE,735-739,< 1 year
1,debt_consolidation,TX,MORTGAGE,715-719,2 years
2,debt_consolidation,CA,MORTGAGE,690-694,2 years
3,debt_consolidation,KS,MORTGAGE,695-699,5 years
4,credit_card,NJ,RENT,695-699,9 years
...,...,...,...,...,...
2495,debt_consolidation,NY,MORTGAGE,705-709,8 years
2496,home_improvement,MD,OWN,740-744,10+ years
2497,debt_consolidation,PA,MORTGAGE,680-684,10+ years
2498,major_purchase,NJ,RENT,675-679,5 years


In [47]:
# data preparation for chi-square test
obs_freq=pd.crosstab(loan_data_cat.home_ownership,loan_data_cat.FICO_Range)
# Ho = no relationship
# Ha = there is a relationship among the variables
# CI = 95%
# p-value = 0.05
print('pvalue=', stats.chi2_contingency(obs_freq)[1])


pvalue= 1.202159201024428e-35


In [48]:
#business conclusion
#home ownership are related with each other

### BUSINESS PROBLEM - 2

In [49]:
price_quote=pd.read_csv('E:/DataSetProject4/Price_Quotes.csv')

In [50]:
price_quote

,Order_Number,Barry_Price,Mary_Price
0,1,126,114
1,2,110,118
2,3,138,114
3,4,142,111
4,5,146,129
5,6,136,119
6,7,94,97
7,8,103,104
8,9,140,127
9,10,152,133


#### We would like to assess if there is any difference in the average price quotes provided by Mary and Barry.


In [51]:
print('mean price of Barry:',(price_quote.Barry_Price.mean()))
print('mean price of Mary:',(price_quote.Mary_Price.mean()))

mean price of Barry: 124.33333333333333
mean price of Mary: 114.75


In [52]:
# Ho : u2 = u1
# Ha : u2 > u1

# CI: 95%
# p: 0.05
# perform the test
stats.ttest_rel(price_quote.Barry_Price,price_quote.Mary_Price)


Ttest_relResult(statistic=2.521376510892349, pvalue=0.02840588045242053)

In [53]:
#business conclusion
#avg price difference between mary and barry

##### BUSINESS PROBLEM-3:

In [54]:
treatment_facility = pd.read_csv('E:/DataSetProject4/Treatment_Facility.csv')

In [55]:
treatment_facility 

,Month,Reengineer,Employee_Turnover,VAR4,VAR5
0,1,Prior,0.0000,24.390244,42.682927
1,2,Prior,6.0606,19.354839,25.806452
2,3,Prior,12.1212,35.087719,146.198830
3,4,Prior,3.3333,18.404908,110.429448
4,5,Prior,12.9032,17.964072,23.952096
5,6,Prior,9.6774,41.176471,47.058824
6,7,Prior,11.7647,13.422819,0.000000
7,8,Prior,11.4286,31.250000,25.000000
8,9,Prior,23.0769,17.241379,132.183908
9,10,Prior,15.0000,16.574586,16.574586


In [56]:
treatment_facility.rename(columns={'VAR4':'TRFF','VAR5':'CI'},inplace= True)
treatment_facility

,Month,Reengineer,Employee_Turnover,TRFF,CI
0,1,Prior,0.0000,24.390244,42.682927
1,2,Prior,6.0606,19.354839,25.806452
2,3,Prior,12.1212,35.087719,146.198830
3,4,Prior,3.3333,18.404908,110.429448
4,5,Prior,12.9032,17.964072,23.952096
5,6,Prior,9.6774,41.176471,47.058824
6,7,Prior,11.7647,13.422819,0.000000
7,8,Prior,11.4286,31.250000,25.000000
8,9,Prior,23.0769,17.241379,132.183908
9,10,Prior,15.0000,16.574586,16.574586


##### Determine what effect, if any, the reengineering effort had on the incidence behavioral problems and staff turnover

In [57]:
a1 = treatment_facility.loc [treatment_facility.Reengineer=='Prior','CI']
a2 = treatment_facility.loc [treatment_facility.Reengineer=='Post','CI']


In [58]:
print('mean of prior:',a1.mean())
print('mean of post:', a2.mean())

mean of prior: 53.887906321846145
mean of post: 23.34971927988571


In [59]:
# Ho : u1 = u2
# Ha : u1 > u2

# CI: 95%
# p: 0.05
# perform the test
stats.ttest_ind(a1,a2)

#business conclusion
#with the given data we can't conclude anything

Ttest_indResult(statistic=1.6279144253528646, pvalue=0.12091989189884149)

#### BUSINESS PROBLEM-4

In [60]:
priority_assessment = pd.read_csv('E:/DataSetProject4/Priority_Assessment.csv')
priority_assessment.isnull().sum()

Days        0
Priority    0
dtype: int64

In [61]:
priority_assessment.head(2)

,Days,Priority
0,3.3,High
1,7.9,Medium


In [62]:
s1 = priority_assessment.loc[ priority_assessment.Priority=='High','Days']
s2 = priority_assessment.loc[ priority_assessment.Priority=='Medium','Days']
s3 = priority_assessment.loc[ priority_assessment.Priority=='Low','Days']

In [63]:
# Ho = no relationship
# Ha = there is a relationship among the variables
# CI = 95%
# p-value = 0.05
#perform the operation
stats.f_oneway(s1,s2,s3)


F_onewayResult(statistic=1.812311010076072, pvalue=0.16411459461716182)

In [64]:
#business conclusion
#with the given data we can't conclude anything

### BUSINESS PROBLEM-5


In [65]:
film = pd.read_csv('E:/DataSetProject4/Films.csv')

In [66]:
film

,_rowstate_,Movie,Gender,Marital_Status,Sinage,Parking,Clean,Overall,Age,Income,Hear_About
0,0,Ferris Buellers Day Off,Female,Married,2.0,2.0,2.0,2.0,3.0,1.0,5
1,0,Ferris Buellers Day Off,Female,Single,1.0,1.0,1.0,1.0,2.0,1.0,5
2,0,Ferris Buellers Day Off,Male,Married,2.0,4.0,3.0,2.0,4.0,1.0,5
3,0,Ferris Buellers Day Off,Female,Married,1.0,3.0,2.0,2.0,4.0,1.0,5
4,0,Ferris Buellers Day Off,Female,Married,1.0,1.0,1.0,1.0,3.0,3.0,1
...,...,...,...,...,...,...,...,...,...,...,...
325,0,Old School,2,2,1.0,2.0,1.0,1.0,2.0,1.0,1
326,0,Old School,1,1,2.0,2.0,2.0,2.0,3.0,3.0,5
327,0,Old School,2,1,2.0,1.0,1.0,2.0,2.0,2.0,5
328,0,Old School,1,1,1.0,1.0,1.0,1.0,2.0,1.0,3


In [67]:
film.isna().sum()

_rowstate_         0
Movie              0
Gender             0
Marital_Status     2
Sinage             2
Parking            2
Clean              3
Overall            2
Age                2
Income            16
Hear_About         7
dtype: int64

In [68]:
film.dtypes

_rowstate_          int64
Movie              object
Gender             object
Marital_Status     object
Sinage            float64
Parking           float64
Clean             float64
Overall           float64
Age               float64
Income            float64
Hear_About         object
dtype: object

In [69]:
film.Gender.value_counts()

2         194
1         104
Female     19
Male       13
Name: Gender, dtype: int64

In [70]:
for var in film.columns:
    film[var].fillna(value = film[var].mode()[0], inplace=True)

In [71]:
film.isna().sum()

_rowstate_        0
Movie             0
Gender            0
Marital_Status    0
Sinage            0
Parking           0
Clean             0
Overall           0
Age               0
Income            0
Hear_About        0
dtype: int64

In [72]:
def gender (x):
    if x['Gender'] =='Male':
        return '1'
    elif x['Gender'] == 'Female':
         return '0'
    elif x['Gender'] == '1':
         return '1'
    else :
         return '0'

In [73]:
film.Gender = film.apply(gender, axis = 1)

In [74]:
film.Gender.value_counts()

0    213
1    117
Name: Gender, dtype: int64

In [75]:
def marrital_status(x):
    if x['Marital_Status']=='Married':
        return '1'
    elif x['Marital_Status']=='Single':
        return '0'
    elif x['Marital_Status']=='1':
        return '1'
    else :
        return '0'

In [76]:
film.Marital_Status = film.apply(marrital_status, axis=1)

#### What isthe overall level of customer satisfaction?

In [77]:
film.head(2)

,_rowstate_,Movie,Gender,Marital_Status,Sinage,Parking,Clean,Overall,Age,Income,Hear_About
0,0,Ferris Buellers Day Off,0,1,2.0,2.0,2.0,2.0,3.0,1.0,5
1,0,Ferris Buellers Day Off,0,0,1.0,1.0,1.0,1.0,2.0,1.0,5


In [78]:
cust_sat=film.groupby('Overall')['Overall'].count()

In [111]:
cust_1 = film.groupby('Overall')[['Overall']].count()
cust_1 = cust_1.rename( columns={'Overall': 'count'}).reset_index()
cust_1['satisfied_perc'] = cust_1['count'] *100 /cust_1['count'].sum()
print(cust_1)



   Overall  count  satisfied_perc
0      1.0    151       45.757576
1      2.0    162       49.090909
2      3.0     12        3.636364
3      4.0      1        0.303030
4      5.0      4        1.212121


##### What is the demographic profile of Film on the Rocks patrons?

In [119]:
gender = film.groupby('Gender')[['Gender']].count()
gender = gender.rename(columns={'Gender':'count'})
gender['perc'] = gender['count']*100/gender['count'].sum()
gender



,count,perc
Gender,,
0,213,64.545455
1,117,35.454545


In [120]:
marital_status = film.groupby('Marital_Status')[['Marital_Status']].count()
marital_status = marital_status.rename(columns={'Marital_Status':'count'})
marital_status['perc'] = marital_status['count']*100/marital_status['count'].sum()
marital_status

,count,perc
Marital_Status,,
0,230,69.69697
1,100,30.30303


#### What factors are linked to satisfaction?

In [80]:
# data preparation for chi-square test
obs_freq1=pd.crosstab(film.Sinage,film.Overall)
# Ho = no relationship
# Ha = there is a relationship among the variables
# CI = 95%
# p-value = 0.05
print('pvalue=', stats.chi2_contingency(obs_freq1)[1])

pvalue= 4.53298970003405e-19


In [81]:
# business conclusion
#with increase in sinage rating overall rating increases

In [82]:
# data preparation for chi-square test
obs_freq2=pd.crosstab(film.Parking,film.Overall)
# Ho = no relationship
# Ha = there is a relationship among the variables
# CI = 95%
# p-value = 0.05
print('pvalue=', stats.chi2_contingency(obs_freq2)[1])

pvalue= 4.783162283608494e-40


In [83]:
# business conclusion
#with increase in Parking rating overall rating increases

In [84]:
# data preparation for chi-square test
obs_freq3=pd.crosstab(film.Clean,film.Overall)
# Ho = no relationship
# Ha = there is a relationship among the variables
# CI = 95%
# p-value = 0.05
print('pvalue=', stats.chi2_contingency(obs_freq3)[1])

pvalue= 1.7772535915434025e-19


In [85]:
# business conclusion
#with increase in clean rating overall rating increases

# In what media outlet(s) should the film series be advertised?


In [125]:
hear_about = film.groupby('Hear_About')[['Hear_About']].count()
hear_about = hear_about.rename(columns={'Hear_About':'count'})
hear_about['perc'] = hear_about['count']*100/hear_about['count'].sum()
hear_about = hear_about.sort_values(by='perc',ascending = False,ignore_index=False)
hear_about

,count,perc
Hear_About,,
5,233,70.606061
4,41,12.424242
1,22,6.666667
3,14,4.242424
2,12,3.636364
"2,5",2,0.606061
"3,4",2,0.606061
"1,5",1,0.303030
"3,5",1,0.303030
